In [ ]:
# ETL Extract

# If don't have, install only once
# !pip install kagglehub

import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("snehangsude/audible-dataset")

df = pd.read_csv(os.path.join(path, 'audible_uncleaned.csv'))

In [ ]:
# ETL Transform

# Check first views:
# df.head()
# df.describe()
# df.info()
# df.columns

# Clean text data in Author and Narrator columns
# Remove Writtenby: from the author column
df['author'] = df['author'].str.replace('Writtenby:', '', regex=True)
df['author'] = df['author'].str.replace(r'(?<!^)(?=[A-Z])', ' ', regex=True)
# Remove Narratedby: from the narrator column
df['narrator'] = df['narrator'].str.replace('Narratedby:', '', regex=True)
df['narrator'] = df['narrator'].str.replace(r'(?<!^)(?=[A-Z])', ' ', regex=True)
# df.head(2)

# Clean stars table
# Create separate column for ratings
df['ratings'] = df['stars'].str.extract(r'stars(\d+)\s')
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce').fillna(0).astype(int)
# Create separate column for stars
df['stars out of five'] = df['stars'].str.extract(r'(\d+)\sout')
df['stars out of five'] = df['stars out of five'].fillna(0).astype('float')
df.drop('stars', axis=1, inplace=True)
# df.head(2)
# df['stars out of five'].unique()
# df['ratings'].sample(n=30)

# Clean price table
# Replace the comma with ''
df['price'] = df['price'].str.replace(',', '', regex=True)
# Replace 'Free' with 0
df['price'] = df['price'].str.replace('Free', '0', regex=True)
# Turn price to float
df['price'] = df['price'].astype(float)
# df['price'].sample(n=30)

# Convert releasedate to datetime
df['releasedate'] = pd.to_datetime(df['releasedate'])
# Inspect the dataframe 
# df.info()
# Search the entries in the time column for different spellings of min. Let' try min, mins, minutes
# df['time'].str.contains(r'\b(min|mins|minutes)\b', case=False, na=False)

# Replace hrs, mins, and 'Less than 1 minute'
df['time'] = df['time'].str.replace(r'\b(mins?|minutes?)\b', 'min', case=False, regex=True)
df['time'] = df['time'].str.replace(r'\b(hr?|hrs?)\b', 'hr', case=False, regex=True)
df['time'] = df['time'].str.replace('Less than 1 minute', '1 min', regex=True)
# set(type(x) for x in df['time'])
# df['time'].isna().any()

# Extract the number of hours, turn to integer
df['hours'] = df['time'].str.extract(r'(\d+)\s*hr')
df['hours'] = df['hours'].fillna("0").astype('str')
# Extract the number of minutes, turn to integer
df['mins'] = df['time'].str.extract(r'and\s(\d+)\s*min')
df['mins'] = df['mins'].fillna("0").astype('int')
df['mins'] = df['mins'].apply(lambda x: f"{x:02}")
# Combine hours and minutes into the duration column
df['duration'] = df['hours'].astype('str') + ':' + df['mins'].astype('str')

# Drop redundant columns
df.drop('mins', axis=1, inplace=True)
df.drop('hours', axis=1, inplace=True)
df.drop('time', axis=1, inplace=True)
# Check the results
# df.head()

# Transform prices to USD (multiply times 0.012)
df['price'] = df['price'] * 0.012
# Check the results
# df['price'].sample(n=30)

# Update capitalization in the language column
df['language'] = df['language'].str.capitalize()
# Check the results
# df['language'].sample(n=30)

# Checking the duplicates
# duplicates = df.duplicated().sum()
# print(duplicates)
subset_cols = ['name', 'author', 'narrator', 'duration', 'price']
duplicates = df.duplicated(subset_cols, keep=False)
# print(duplicates.sample(n=30))
# drop duplicates
unduped_df = df.drop_duplicates()

# Check if there is any Nan
# print(df.isna().sum())
df.head(2)

In [ ]:
# ETL Load

# Save to csv
# df.to_csv('audible.csv', index=False)